# Introdução

*Script* que cria uma tabela que apresenta informações das *Coordenadorias Regionais de Proteção e Defesa Civil - **REPDEC*** da Defesa Civil que atuam nos 645 município deo Estado de São Paulo.

In [ ]:
import os
import time
import random
import shutil
import requests

import numpy as np
import urllib.parse
import pandas as pd
import geopandas as gpd

from osgeo import ogr
from bs4 import BeautifulSoup
from zipfile import ZipFile
from pdf2jpg import pdf2jpg
from osgeo import gdal, osr
from tqdm.notebook import trange, tqdm
from PyPDF2 import PdfFileReader, PdfFileWriter

In [ ]:
#!pip install PyPDF2
#!pip install pdf2jpg

# Dados Espaciais: PDF

## *Download*

Inicialmente foi feito o download da [monografia](http://www.sidec.sp.gov.br/defesacivil/media/OSDownloads/1398863063_Monografia%20Cap%20PM%20Rudyard.pdf)  desenvolvida pelo Capitão da PM, Rudyard Panzarini Paiva, em 2013, com o título *'Regulamentação dos Critérios para Homologação das Situações de Anormalidade no Estado de São Paulo'*, elaborado para concluir o 'Curso de Aperfeiçoamento de Oficiais – II/2012'.

Na monografia, na página 33, é apresentado o mapa com as áreas de abrangência de cada que Indicações sobre a área de atuação de cada Coordenadorias Regionais de Proteção e Defesa Civil - REPDEC, além disso é informado que:

```
As jurisdições das Coordenadorias Regionais de Proteção e Defesa Civil acompanham as Regiões Administrativas do Estado de São Paulo, porém majoradas para 19, pois foram acrescentadas as regiões do ABCDMRR, Guarulhos e Osasco.
```

Pesquisando informaçãoes sobre a sigla mencionada, encontrou que:
```
ABC Paulista, Região do Grande ABC, ABC ou ainda ABCD, é uma região tradicionalmente industrial do estado de São Paulo, parte da Região Metropolitana de São Paulo, porém com identidade própria. A sigla vem das três cidades que, originalmente, formavam a região, sendo: Santo André (A), São Bernardo do Campo (B) e São Caetano do Sul (C). Às vezes, Diadema (D) é incluída na sigla.

É relativamente comum encontrar também ABCDMRR que também inclui os municípios de Mauá, Ribeirão Pires e Rio Grande da Serra.
```

In [ ]:
def download_urls(urls, path, GetFilenameFromURL=True):
    """
    Function to download list of files with a progress bar.
    :param urls: Lista ou tuple, a depender da função #
    :param path: Local onde os arquivos serão inseridos
    :param GetFilenameFromURL: Se VERDADEIRO, a função irá nomear o arquivo conforme o link das lista das URLs.
    Se FALSO, a função irá nomear o arquivo conforme string definida no tuple das URLs
    """
    
    # Reset Interactions
    i = 0
    n_urls = len(urls)

    for n_url in trange(n_urls, desc='Total'):
        # Download path and file name
        if GetFilenameFromURL:
            url = urls[i]
            filename = urls[i].rsplit('/', 1)[1]
        else:
            url = urls[i][0]
            filename = urls[i][1]

        # File size
        r = requests.get(url, stream=True)
        chunk_size = 1024 * 1024
        total_size = int(r.headers['content-length'])

        # Download the file from 'url' and save it locally under 'filename'
        with open(os.path.join(path, filename), 'wb') as f:
            for data in tqdm(
                iterable = r.iter_content(chunk_size=chunk_size),
                total = int(total_size/chunk_size),
                unit = 'MB',
                desc = '{}/{}'.format(str(i+1), str(n_urls))):
                f.write(data)

        # Interactions
        i = i+1

        # Definir um intervalo de tempo
        time.sleep(random.randint(1, 3))

In [ ]:
url = 'http://www.sidec.sp.gov.br/defesacivil/media/OSDownloads/1398863063_Monografia%20Cap%20PM%20Rudyard.pdf'
filename= '2013 - Regulamentação dos critérios.pdf'
tuple_pdf = [(url, filename)]

In [ ]:
download_urls(tuple_pdf, 'docs', GetFilenameFromURL=False)

## Recortar mapa do PDF

Uma vez com o arquivo *pdf*, selecionei apenas a página que me interessava.

In [ ]:
# I/O
in_filename = os.path.join('docs', '2013 - Regulamentação dos critérios.pdf')
out_filename = os.path.join('docs', 'temp.pdf')

# Read File
in_pdf = PdfFileReader(in_filename)

# Select Pages
pages = [33]

# Create Object to output
out_pdf = PdfFileWriter()

# Loop trough pages
for page in pages:
    out_pdf.addPage(in_pdf.getPage(page))

# Save
with open(out_filename, 'wb') as f:
    out_pdf.write(f)
    f.close()

<br>

E recortei o trecho que me interessava

In [ ]:
# I/O
in_filename = os.path.join('docs', 'temp.pdf')
out_filename = os.path.join('docs', 'mapa.pdf')

with open(in_filename, 'rb') as f:
    in_pdf = PdfFileReader(f)
    out_pdf = PdfFileWriter()

    numPages = in_pdf.getNumPages()
    print('document has {} pages.'.format(numPages))

    for i in range(numPages):
        page = in_pdf.getPage(i)
        print(
            page.mediaBox.getUpperRight_x(),
            page.mediaBox.getUpperRight_y()
        )        
        #page.trimBox.lowerLeft = (0, 0)
        #page.trimBox.upperRight = (525, 625)
        page.cropBox.lowerLeft = (130, 380)
        page.cropBox.upperRight = (530, 700)
        out_pdf.addPage(page)

    with open(out_filename, 'wb') as out_f:
        out_pdf.write(out_f)

<br>

Posteriormente converti para imagem.

In [ ]:
# to convert all pages
result = pdf2jpg.convert_pdf2jpg(
    os.path.join('docs', 'mapa.pdf'),
    os.path.join('docs'),
    pages='ALL')

print(result)

<br>

Renomeie

In [ ]:
os.rename(
    result[0]['output_jpgfiles'][0],
    os.path.join('data', 'rasters', 'mapa.jpg')    
)

<br>

E deletei os arquivos temporários.

In [ ]:
os.remove(in_filename)
os.remove(out_filename)
shutil.rmtree(os.path.dirname(result[0]['output_jpgfiles'][0]))

<br>

## Georeferenciamento

Uma vez com o mapa em mãos, em *jpg*, fiz o georreferenciamento dele.

In [ ]:
def georeferencing(src_filename, dst_filename, gcp_list):
    # 
    src_ds = gdal.Open(src_filename, gdal.GA_ReadOnly)
    driver = gdal.GetDriverByName('JPEG')

    # Open destination dataset
    dst_ds = driver.CreateCopy(dst_filename, src_ds, 0)

    # Get raster projection
    srs = osr.SpatialReference()
    srs.ImportFromEPSG(4674)
    dst_wkt = srs.ExportToWkt()

    # Apply the GCPs to the open output file:
    dst_ds.SetGCPs(gcp_list, dst_wkt)

    # Warp Image
    dst_ds = gdal.Warp(dst_filename, dst_ds, dstSRS=dst_wkt)

    # Close files
    src_ds = None
    dst_ds = None
    
    print('Arquivo "{}" georreferenciado.'.format(src_filename))

In [ ]:
# Interior
gcp = []
gcp.append(gdal.GCP(-53.08776489000207732,-22.65848512212706112,0,67.39315671031094723,734.81352291325697479))
gcp.append(gdal.GCP(-44.1613485118776623,-22.67839016588562018,0,1613.78769435353024164,693.92426605975663279))
gcp.append(gdal.GCP(-48.89569728207362687,-20.44167357629228476,0,784.82150163667267861,196.14571399347153147))
gcp.append(gdal.GCP(-48.09945589998689996,-25.31233573226733213,0,943.74248158757416149,1298.82619425124539703))
gcp.append(gdal.GCP(-46.64917183567540349,-21.36317108739623194,0,1179.87410495091262419,396.53545673078855316))
gcp.append(gdal.GCP(-51.77248636389690262,-21.05130888861864591,0,284.51187218700169979,359.08062474429112854))

georeferencing(
    os.path.join('data', 'rasters', 'mapa.jpg'),
    os.path.join('data', 'rasters', 'geo_defesacivil.jpg'),
    gcp,
)

<br>

## Spatial Join

Usando o código *get_spatial_data.ipynb* foi possível baixar a malha municipal do estado de São Paulo do [*site* do IBGE](https://geoftp.ibge.gov.br/organizacao_do_territorio/malhas_territoriais/malhas_municipais/municipio_2020/UFs/SP/SP_Municipios_2020.zip). Nos códigos abaixo o arquivo *shapefile* é lido, são feitos pequenos ajustes para ter um material mais "limpo" para definir, manualmente, quais os "Grupamentos de Bombeiro" que atual em cada um dos municípios.

In [ ]:
# Lê o arquivo shapefile
file = 'sp_ibge_250k'
shp = os.path.join('data', 'shps', '{}.shp'.format(file))
gdf = gpd.read_file(shp)

# Deleta colunas
gdf.drop(['SIGLA_UF', 'AREA_KM2'], axis=1, inplace=True)

# Renomeia colunas
gdf.rename(columns={'CD_MUN':'id_mun'}, inplace=True)
gdf.rename(columns={'NM_MUN':'municipio'}, inplace=True)

# Reordena colunas
gdf['repdec_cod'] = 'I'
gdf = gdf[['id_mun', 'municipio', 'repdec_cod', 'geometry']]

# ATENÇÃO: SÓ EXECUTAR SE FOR REFAZER A CLASSIFICAÇÃO MANUALMENTE NO QGIS!
#gdf.to_file(os.path.join('data', 'shps', 'div_admin.shp'), encoding='utf-8')

# Results
gdf.head()

<br>

# Dados Espaciais: DataGeo

In [ ]:
def get_download(url, filename, path):
    # Define o nome do arquivo, com diretório, que será salvo
    zipfile = os.path.join(path, filename)

    # Faz o download do arquivo da 'url' e salva localmente com o nome do arquivo
    r = requests.get(url, stream=True)
    with open(zipfile, 'wb') as out:
        for chunk in r.iter_content(chunk_size=128):
            out.write(chunk)
    print('File "{}" download in "{}" directory.'.format(filename, path))

In [ ]:
def unzip(filename, path):
    # Get dir of zipfile
    zipfile = os.path.join(path, filename)
    path = os.path.dirname(zipfile)

    # Create a ZipFile Object
    with ZipFile(zipfile, 'r') as zip_obj:
        # Get a list of all archived file names from the zip
        list_files = []
        list_files = zip_obj.namelist()

        # Iterate over the file names
        for file in list_files:
            name, ext = os.path.splitext(file)
            zip_obj.extract(file, os.path.dirname(zipfile))
            os.rename(os.path.join(path, file), os.path.join(path, os.path.splitext(filename)[0]+ext))
            print('File "{}" extracted as "{}" in "{}" directory.'.format(file, os.path.splitext(filename)[0]+ext, path))

    # Remove file
    os.remove(zipfile)

In [ ]:
def rename_nome(x):
    x = x.title()
    x = x.strip()
    dict_rename = {
        '/': '-',
        ' Do ': ' do ',
        ' Dos ': ' dos ',
    }
    for k, v in dict_rename.items():
        x = x.replace(k, v)
    x = x.replace('  ', ' ')
    return x.strip()

<br>

## *Download*

In [ ]:
# Input
url = 'http://datageo.ambiente.sp.gov.br/geoserver/datageo/RegionaisdaDefesaCivil/wfs?version=1.0.0&request=GetFeature&outputFormat=SHAPE-ZIP&typeName=RegionaisdaDefesaCivil'
filename = 'sp_defesacivil.zip'
path = os.path.join('data', 'shps')

# Get Data
get_download(url, filename, path)
unzip(filename, path)

In [ ]:
# Lê arquivo
file = 'sp_defesacivil'
shp = os.path.join('data', 'shps', '{}.shp'.format(file))
gdf = gpd.read_file(shp)

# Transforma Coordenadas
gdf = gdf.to_crs(epsg=4326)

# Renomeia colunas
gdf.rename(
    columns={
        'Codigo': 'repdec_cod',
        'Nome'  : 'repdec_regiao',
    },
    inplace=True
)

# Renomeia Campos Coluna
gdf.loc[:, 'repdec_cod'] = gdf['repdec_cod'].astype(str).apply(lambda x: rename_nome(x))
gdf.loc[:, 'repdec_regiao'] = gdf['repdec_regiao'].astype(str).apply(lambda x: rename_nome(x))

# Results
display(gdf.head())
#gdf.plot()

# Cria Cópia
gdf_repdec = gdf

In [ ]:
# Deleta Shapefile
file = 'sp_defesacivil'
shp = os.path.join('data', 'shps', '{}.shp'.format(file))
driver = ogr.GetDriverByName("ESRI Shapefile")

# Delete shapefile
if os.path.exists(shp):
    driver.DeleteDataSource(shp)

# Delete other files
for i in os.listdir(os.path.dirname(shp)):
    if i.split('.')[0] == file:
        os.remove(os.path.join('data', 'shps', i))

<br>

## Convert to Points

In [ ]:
# Lê arquivo
gdf = gpd.read_file(
    'https://raw.githubusercontent.com/michelmetran/sp/main/data/shps/sp_050k_wgs84.geojson',
)

# Transforma Coordenadas
gdf = gdf.to_crs(epsg=4326)

# Pega centroid
gdf_geometry = gdf.representative_point()
gdf['geometry'] = gdf_geometry

# Results
display(gdf.head())
gdf.plot()

<br>

## Spatial Join

In [ ]:
# Intersect
gdf_inter = gpd.sjoin(gdf, gdf_repdec, how='inner', op='intersects')

# Convert to Dataframe
df = gdf_inter[['id_municipio', 'repdec_cod', 'repdec_regiao']].copy()

# Results
display(df.head())
gdf_inter.plot()

<br>

## Join Municípios

In [ ]:
# Lê o arquivo csv com o nome dos municípios
df_mun = pd.read_csv(
    'https://raw.githubusercontent.com/michelmetran/sp/main/data/tabs/tab_municipios.csv',
    usecols=['id_municipio', 'municipio_nome']
)

# Merge
df = pd.merge(
    df_mun,
    df,
    how='left',
    left_on='id_municipio',
    right_on='id_municipio'
)

# Resultados
df.head()

<br>

## Join Defesa Civil

Por meio do [*site* da Defesa Civil](http://www.defesacivil.sp.gov.br/) (mais específicamente nesse [*link*](http://www.defesacivil.sp.gov.br/coordenadores-regionais-de-defesa-civil/)) foi possível obter diversas informações sobre os Coordenadores que atuam nas 'Coordenadorias Regionais de Proteção e Defesa Civil - REPDEC'.

Foi utilizado o *script* **get_infos.ipynb** para isso. Abaixo o arquivo gerado é apenas lido, para ser conctenado com a lista de municípios.

In [ ]:
# Lê o arquivo csv com o nome dos municípios
df_defesa = pd.read_csv(
    os.path.join('data', 'tabs', 'tab_defesacivil.csv'),
)

# Merge
df = pd.merge(
    df,
    df_defesa,    
    how='left',
    left_on='repdec_cod',
    right_on='repdec_cod'
)

# Resultados
df.head()

<br>

## Salva

In [ ]:
# Salva
df.to_csv(
    os.path.join('data', 'tabs', 'tab_municipio_defesacivil.csv'),
    index=False,
)

# Resultados
df.head()